In [ ]:
# !pip install pmdarima

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import os
import geopandas as gpd
import seaborn as sns

# suppress warning
import warnings
warnings.filterwarnings('ignore')

import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
import scipy
from sklearn.metrics import r2_score
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import missingno as msno
from sklearn.feature_selection import SelectFromModel
import xgboost as xgb
import os
import io

from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, CyclicLR
import torch.nn.functional as F
# from pmdarima import auto_arima
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split, KFold
#sns.set()

%matplotlib inline

In [3]:
# # load eviction data
# from google.colab import files
# uploaded = files.upload()

# uploaded manually

In [9]:
# evictions = pd.read_csv(io.BytesIO(uploaded['Evictions_20240918.csv']))
evictions_df = pd.read_csv('evictions.csv')

In [10]:
evictions_df.head(1)

,Court Index Number,Docket Number,Eviction Address,Eviction Apartment Number,Executed Date,Marshal First Name,Marshal Last Name,Residential/Commercial,BOROUGH,Eviction Postcode,Ejectment,Eviction/Legal Possession,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,315863/23,16461,626 RIVERSIDE DRIVE,18O,02/08/2024,David,Smith,Residential,MANHATTAN,10031,Not an Ejectment,Possession,40.823647,-73.954431,9.0,7.0,225.0,1062307.0,1.020870e+09,Hamilton Heights


In [ ]:
# !pip install openpyxl

In [22]:
complaints_df = pd.read_csv('small_311.csv')

In [23]:
complaints_df.info(), \
complaints_df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300698 entries, 0 to 300697
Data columns (total 53 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Unique Key                      300698 non-null  int64  
 1   Created Date                    300698 non-null  object 
 2   Closed Date                     298534 non-null  object 
 3   Agency                          300698 non-null  object 
 4   Agency Name                     300698 non-null  object 
 5   Complaint Type                  300698 non-null  object 
 6   Descriptor                      294784 non-null  object 
 7   Location Type                   300567 non-null  object 
 8   Incident Zip                    298083 non-null  float64
 9   Incident Address                256288 non-null  object 
 10  Street Name                     256288 non-null  object 
 11  Cross Street 1                  251419 non-null  object 
 12  Cross Street 2  

(None, (300698, 53))

In [24]:
complaints_df.head(1)

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,32310363,12/31/2015 11:59:45 PM,01-01-16 0:55,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Music/Party,Street/Sidewalk,10034.0,71 VERMILYEA AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.865682,-73.923501,"(40.86568153633767, -73.92350095571744)"


In [20]:
BBL_df = pd.read_csv('BBL.csv')

In [21]:
BBL_df.head(2)

,borough,block,lot,community board,census tract 2010,cb2010,schooldist,council district,postcode,firecomp,...,dcasdate,zoningdate,landmkdate,basempdate,masdate,polidate,edesigdate,geom,dcpedited,notes
0,BX,2587,22,201.0,19.0,3026.0,7.0,17.0,10454.0,L029,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MN,135,14,101.0,21.0,1002.0,2.0,1.0,10007.0,L010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
len(complaints_df)

300698

In [26]:
complaints_df.columns

Index(['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name',
       'Complaint Type', 'Descriptor', 'Location Type', 'Incident Zip',
       'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Address Type',
       'City', 'Landmark', 'Facility Type', 'Status', 'Due Date',
       'Resolution Description', 'Resolution Action Updated Date',
       'Community Board', 'Borough', 'X Coordinate (State Plane)',
       'Y Coordinate (State Plane)', 'Park Facility Name', 'Park Borough',
       'School Name', 'School Number', 'School Region', 'School Code',
       'School Phone Number', 'School Address', 'School City', 'School State',
       'School Zip', 'School Not Found', 'School or Citywide Complaint',
       'Vehicle Type', 'Taxi Company Borough', 'Taxi Pick Up Location',
       'Bridge Highway Name', 'Bridge Highway Direction', 'Road Ramp',
       'Bridge Highway Segment', 'Garage Lot Name', 'Ferry 

In [29]:
complaints_df["Due Date"] = pd.to_datetime(complaints_df["Due Date"])

In [30]:
complaints_df["Due Date"].min(), complaints_df["Due Date"].max()

(Timestamp('2015-03-29 08:33:01'), Timestamp('2016-01-01 07:59:00'))

In [31]:
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/My Drive/X999/last_one_tenth.csv'

Mounted at /content/drive


In [33]:
small_complaints_df = pd.read_csv(file_path)

In [37]:
small_complaints_df.tail(2)

,43870781,2019-09-23T12:00:00.000,2019-09-23T21:35:00.000,DEP,Department of Environmental Protection,Water System,Hydrant Running Full (WA4),Unnamed: 7,11429,Unnamed: 9,...,Unnamed: 36,Unnamed: 37,40.71036230723576,-73.7299822118594,"{'longitude': '-73.7299822118594', 'latitude': '40.71036230723576', 'human_address': '{""address"": """", ""city"": """", ""state"": """", ""zip"": """"}'}",24338,63,3,6,63.1
2195997,43867439,2019-09-22T11:38:28.000,2019-09-22T12:16:31.000,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,11207.0,995 BARBEY STREET,...,NaN,NaN,40.658186,-73.880636,"{'longitude': '-73.88063634925773', 'latitude'...",17214.0,45.0,2.0,25.0,47.0
2195998,43861543,2019-09-22T11:38:12.000,NaN,DPR,Department of Parks and Recreation,Damaged Tree,Tree Alive - in Poor Condition,Street,11362.0,60-23 246 PLACE,...,NaN,NaN,40.757501,-73.731130,"{'longitude': '-73.73113027986656', 'latitude'...",14197.0,26.0,3.0,16.0,69.0


In [38]:
small_complaints_df.shape

(2195999, 46)

In [40]:
small_complaints_df.tail()

,43870781,2019-09-23T12:00:00.000,2019-09-23T21:35:00.000,DEP,Department of Environmental Protection,Water System,Hydrant Running Full (WA4),Unnamed: 7,11429,Unnamed: 9,...,Unnamed: 36,Unnamed: 37,40.71036230723576,-73.7299822118594,"{'longitude': '-73.7299822118594', 'latitude': '40.71036230723576', 'human_address': '{""address"": """", ""city"": """", ""state"": """", ""zip"": """"}'}",24338,63,3,6,63.1
2195994,43863796,2019-09-22T11:39:38.000,2019-09-22T20:09:28.000,NYPD,New York City Police Department,Blocked Driveway,Partial Access,Street/Sidewalk,11416.0,89-17 102 ROAD,...,NaN,NaN,40.681553,-73.850258,"{'longitude': '-73.85025819596127', 'latitude'...",24012.0,46.0,3.0,41.0,60.0
2195995,43865372,2019-09-22T11:39:25.000,2019-09-26T13:02:49.000,DPR,Department of Parks and Recreation,Overgrown Tree/Branches,Hitting Building,Street,10467.0,25 EAST MOSHOLU PARKWAY NORTH,...,NaN,NaN,40.879794,-73.883435,"{'longitude': '-73.88343522736254', 'latitude'...",11605.0,24.0,5.0,40.0,34.0
2195996,43868018,2019-09-22T11:39:20.000,NaN,TLC,Taxi and Limousine Commission,For Hire Vehicle Complaint,Driver Complaint - Non Passenger,NaN,10013.0,231 HUDSON STREET,...,NaN,NaN,40.724282,-74.007840,"{'longitude': '-74.00783967560663', 'latitude'...",12076.0,57.0,4.0,10.0,1.0
2195997,43867439,2019-09-22T11:38:28.000,2019-09-22T12:16:31.000,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,11207.0,995 BARBEY STREET,...,NaN,NaN,40.658186,-73.880636,"{'longitude': '-73.88063634925773', 'latitude'...",17214.0,45.0,2.0,25.0,47.0
2195998,43861543,2019-09-22T11:38:12.000,NaN,DPR,Department of Parks and Recreation,Damaged Tree,Tree Alive - in Poor Condition,Street,11362.0,60-23 246 PLACE,...,NaN,NaN,40.757501,-73.731130,"{'longitude': '-73.73113027986656', 'latitude'...",14197.0,26.0,3.0,16.0,69.0


In [45]:
file_path = '/content/drive/My Drive/X999/311_complaints.csv'
print(file_path)

/content/drive/My Drive/X999/311_complaints.csv


In [47]:
# big_complaints_df = pd.read_csv(file_path)
# # 13.68 GB file actually loaded on google colab....
# # tried, not working

In [48]:
total_rows = sum(1 for row in open(file_path))
start_row = total_rows * 4 // 5
df = pd.read_csv(file_path, skiprows=range(1, start_row))

# could have done in command lines, but to make it more explicit, use python here.


   Unique Key             Created Date              Closed Date Agency  \
0    43862472  2019-09-22T19:13:52.000  2019-09-22T20:48:28.000   NYPD   
1    43862032  2019-09-22T19:13:29.000  2019-09-22T23:03:56.000   NYPD   
2    43863338  2019-09-22T19:13:25.000  2019-09-23T01:34:53.000   NYPD   
3    43864544  2019-09-22T19:13:10.000  2019-09-22T20:45:41.000   NYPD   
4    43862615  2019-09-22T19:12:56.000  2019-09-25T11:26:00.000    TLC   

                       Agency Name       Complaint Type          Descriptor  \
0  New York City Police Department  Noise - Residential    Loud Music/Party   
1  New York City Police Department      Illegal Parking     Blocked Hydrant   
2  New York City Police Department    Abandoned Vehicle  With License Plate   
3  New York City Police Department     Blocked Driveway           No Access   
4    Taxi and Limousine Commission        Lost Property          Bag/Wallet   

                Location Type  Incident Zip            Incident Address  ...  \


In [52]:
# df contains only the last 1/5 of the file
complaints_df = df
complaints_df.tail()

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
4391996,43863796,2019-09-22T11:39:38.000,2019-09-22T20:09:28.000,NYPD,New York City Police Department,Blocked Driveway,Partial Access,Street/Sidewalk,11416.0,89-17 102 ROAD,...,NaN,NaN,40.681553,-73.850258,"{'longitude': '-73.85025819596127', 'latitude'...",24012.0,46.0,3.0,41.0,60.0
4391997,43865372,2019-09-22T11:39:25.000,2019-09-26T13:02:49.000,DPR,Department of Parks and Recreation,Overgrown Tree/Branches,Hitting Building,Street,10467.0,25 EAST MOSHOLU PARKWAY NORTH,...,NaN,NaN,40.879794,-73.883435,"{'longitude': '-73.88343522736254', 'latitude'...",11605.0,24.0,5.0,40.0,34.0
4391998,43868018,2019-09-22T11:39:20.000,NaN,TLC,Taxi and Limousine Commission,For Hire Vehicle Complaint,Driver Complaint - Non Passenger,NaN,10013.0,231 HUDSON STREET,...,NaN,NaN,40.724282,-74.007840,"{'longitude': '-74.00783967560663', 'latitude'...",12076.0,57.0,4.0,10.0,1.0
4391999,43867439,2019-09-22T11:38:28.000,2019-09-22T12:16:31.000,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,11207.0,995 BARBEY STREET,...,NaN,NaN,40.658186,-73.880636,"{'longitude': '-73.88063634925773', 'latitude'...",17214.0,45.0,2.0,25.0,47.0
4392000,43861543,2019-09-22T11:38:12.000,NaN,DPR,Department of Parks and Recreation,Damaged Tree,Tree Alive - in Poor Condition,Street,11362.0,60-23 246 PLACE,...,NaN,NaN,40.757501,-73.731130,"{'longitude': '-73.73113027986656', 'latitude'...",14197.0,26.0,3.0,16.0,69.0


In [53]:
complaints_df.columns

Index(['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name',
       'Complaint Type', 'Descriptor', 'Location Type', 'Incident Zip',
       'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Address Type',
       'City', 'Landmark', 'Facility Type', 'Status', 'Due Date',
       'Resolution Description', 'Resolution Action Updated Date',
       'Community Board', 'BBL', 'Borough', 'X Coordinate (State Plane)',
       'Y Coordinate (State Plane)', 'Open Data Channel Type',
       'Park Facility Name', 'Park Borough', 'Vehicle Type',
       'Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name',
       'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment',
       'Latitude', 'Longitude', 'Location', 'Zip Codes', 'Community Districts',
       'Borough Boundaries', 'City Council Districts', 'Police Precincts'],
      dtype='object')

In [54]:
complaints_df["Created Date"] = pd.to_datetime(complaints_df["Created Date"])
complaints_df["Closed Date"] = pd.to_datetime(complaints_df["Closed Date"])

In [56]:
complaints_df["Created Date"].min(), complaints_df["Created Date"].max(), \
complaints_df["Closed Date"].min(), complaints_df["Closed Date"].max()

(Timestamp('2019-09-22 11:38:12'),
 Timestamp('2019-09-24 02:45:20'),
 Timestamp('2019-09-20 11:08:00'),
 Timestamp('2019-11-29 15:28:10'))

In [57]:
complaints_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4392001 entries, 0 to 4392000
Data columns (total 46 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   Unique Key                      int64         
 1   Created Date                    datetime64[ns]
 2   Closed Date                     datetime64[ns]
 3   Agency                          object        
 4   Agency Name                     object        
 5   Complaint Type                  object        
 6   Descriptor                      object        
 7   Location Type                   object        
 8   Incident Zip                    float64       
 9   Incident Address                object        
 10  Street Name                     object        
 11  Cross Street 1                  object        
 12  Cross Street 2                  object        
 13  Intersection Street 1           object        
 14  Intersection Street 2           object        
 15

In [58]:
complaints_df.shape

(4392001, 46)

In [60]:
complaints_df.to_csv('/content/drive/My Drive/X999/last_fifth.csv', index=False)


In [63]:
BBL_df.columns

Index(['borough', 'block', 'lot', 'community board', 'census tract 2010',
       'cb2010', 'schooldist', 'council district', 'postcode', 'firecomp',
       'policeprct', 'healtharea', 'sanitboro', 'sanitsub', 'address',
       'zonedist1', 'zonedist2', 'zonedist3', 'zonedist4', 'overlay1',
       'overlay2', 'spdist1', 'spdist2', 'spdist3', 'ltdheight', 'splitzone',
       'bldgclass', 'landuse', 'easements', 'ownertype', 'ownername',
       'lotarea', 'bldgarea', 'comarea', 'resarea', 'officearea', 'retailarea',
       'garagearea', 'strgearea', 'factryarea', 'otherarea', 'areasource',
       'numbldgs', 'numfloors', 'unitsres', 'unitstotal', 'lotfront',
       'lotdepth', 'bldgfront', 'bldgdepth', 'ext', 'proxcode', 'irrlotcode',
       'lottype', 'bsmtcode', 'assessland', 'assesstot', 'exempttot',
       'yearbuilt', 'yearalter1', 'yearalter2', 'histdist', 'landmark',
       'builtfar', 'residfar', 'commfar', 'facilfar', 'borocode', 'bbl',
       'condono', 'tract2010', 'xcoord', 'y

In [64]:
evictions_df.columns

Index(['Court Index Number', 'Docket Number ', 'Eviction Address',
       'Eviction Apartment Number', 'Executed Date', 'Marshal First Name',
       'Marshal Last Name', 'Residential/Commercial', 'BOROUGH',
       'Eviction Postcode', 'Ejectment', 'Eviction/Legal Possession',
       'Latitude', 'Longitude', 'Community Board', 'Council District',
       'Census Tract', 'BIN', 'BBL', 'NTA'],
      dtype='object')

In [65]:
file_path = '/content/drive/My Drive/X999/last_one_tenth.csv'
print(file_path)

/content/drive/My Drive/X999/last_one_tenth.csv


In [66]:
last_tenth_complaints = pd.read_csv(file_path)

In [67]:
last_tenth_complaints.shape

(2195999, 46)

In [71]:
# last_tenth_complaints.columns =

In [70]:
# last_tenth_complaints["Created Date"] = pd.to_datetime(last_tenth_complaints["Created Date"])
# last_tenth_complaints["Closed Date"] = pd.to_datetime(last_tenth_complaints["Closed Date"])